In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [ ]:
from typing import List
from summac.model_summac import SummaCZS, SummaCConv
import jsonlines
import nltk
nltk.download('punkt_tab')


metamorphic = ['err_nli', 'err_translate', 'nega_exp', 'num_sub', 'obj_sub', 'verb_sub', 'word_del', 'word_swap']
distance_metrics = ['cosine', 'euclidean', 'person', 'manhattan', 'lancewilliams', 'mahalanobis', 'braycurtis']



def load_dataset(path) -> List[List[str]]:
    data = []
    with jsonlines.open(path) as f:
        for line in f:
            data.append([line['sentence1'], line['sentence2'], line['sentence3']])
    return data






/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [ ]:
import pandas as pd
import torch
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device=device, start_file="default", agg="mean")


def load_dataset(path):
    df = pd.read_json(path, lines=True)
    return df[['sentence1', 'sentence2', 'sentence3']].values.tolist()

batch_size = 32

scores_pos = []
scores_neg = []
for me in metamorphic:
  dataset = load_dataset('data/MeTMaP/dataset/normal/'+me+'.jsonl')
  score_neg = []
  score_pos = []
  for i in range(0, len(dataset), batch_size):
      batch_base = [data[0] for data in dataset[i:i + batch_size]]
      batch_pos = [data[1] for data in dataset[i:i + batch_size]]
      batch_neg = [data[2] for data in dataset[i:i + batch_size]]

      scores_pos.extend(model_conv.score(batch_base, batch_pos)['scores'])
      scores_neg.extend(model_conv.score(batch_base, batch_neg)['scores'])
      print(i)
  score = [scores_pos, scores_neg]
  np.save('SummaCConv' + me + '.npy', score)



In [ ]:
for me in metamorphic:
  dataset = load_dataset('data/MeTMaP/dataset/normal/'+me+'.jsonl')
  score_neg = []
  score_pos = []
  for i in range(0, len(dataset), batch_size):
      batch_base = [data[0] for data in dataset[i:i + batch_size]]
      batch_pos = [data[1] for data in dataset[i:i + batch_size]]
      batch_neg = [data[2] for data in dataset[i:i + batch_size]]

      scores_pos.extend(model_conv.score(batch_pos, batch_base)['scores'])
      scores_neg.extend(model_conv.score(batch_neg, batch_base)['scores'])
      print(i)
  score = [scores_pos, scores_neg]
  np.save('SummaCConv_reverse' + me + '.npy', score)

In [ ]:
model_zs = SummaCZS(granularity="sentence", model_name="vitc", device=device)
for me in metamorphic:
  dataset = load_dataset('data/MeTMaP/dataset/normal/'+me+'.jsonl')
  score_neg = []
  score_pos = []
  for i in range(0, len(dataset), batch_size):
      batch_base = [data[0] for data in dataset[i:i + batch_size]]
      batch_pos = [data[1] for data in dataset[i:i + batch_size]]
      batch_neg = [data[2] for data in dataset[i:i + batch_size]]

      scores_pos.extend(model_zs.score(batch_base, batch_pos)['scores'])
      scores_neg.extend(model_zs.score(batch_base, batch_neg)['scores'])
      print(i)
  score = [scores_pos, scores_neg]
  np.save('SummaCZS_' + me + '.npy', score)

In [ ]:
scores_pos = []
scores_neg = []
for me in metamorphic:
  dataset = load_dataset('data/MeTMaP/dataset/normal/'+me+'.jsonl')
  score_neg = []
  score_pos = []
  for i in range(0, len(dataset), batch_size):
      batch_base = [data[0] for data in dataset[i:i + batch_size]]
      batch_pos = [data[1] for data in dataset[i:i + batch_size]]
      batch_neg = [data[2] for data in dataset[i:i + batch_size]]

      scores_pos.extend(model_zs.score(batch_pos, batch_base)['scores'])
      scores_neg.extend(model_zs.score(batch_neg, batch_base)['scores'])
      print(i)
  score = [scores_pos, scores_neg]
  np.save('SummaCZS_Reverse' + me + '.npy', score)